In [45]:
import glob
from natsort import natsorted
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from typing import Final
from __future__ import annotations

In [46]:
RACE_RESULTS_HEADER: Final[tuple[str]] = (
    '日付','開催場所','ラウンド','グレード','レースグループ','レース名','発走時間','距離','周回',
    '2車複','2車単','ワイド','3連複','3連単'
)
RACE_RESULTS_HEADER_RENAME: Final[tuple[str]] = (
    '日付','開催場所','グレード','レースグループ','レース名','発走時間',
    '2車複','2車単','ワイド','3連複','3連単','ラウンド','距離','周回','発走日時','1着', '2着'
)
RACE_RESULTS_CONV_HEADER: Final[tuple[str]] = (
    'ラウンド','距離','周回'
)
LABELENCODER_COLUMNS: Final[tuple[str]] = (
    '開催場所', 'ラウンド', 'グレード', 'レース名'
)

In [47]:
def get_df() -> pd.DataFrame:
    files = natsorted(glob.glob('race_data/race_results*.csv'))

    race_results_list = []
    for file in files:
        try:
            race_results_list.append(pd.read_csv(file, encoding='cp932', header=0, names=RACE_RESULTS_HEADER))
        except UnicodeDecodeError:
            race_results_list.append(pd.read_csv(file, encoding='utf-8', header=0, names=RACE_RESULTS_HEADER))
    df_race_results = pd.concat(race_results_list)
    
    return df_race_results

In [48]:
def char_exclusion(df: pd.DataFrame) -> pd.DataFrame:
    for column_name in RACE_RESULTS_CONV_HEADER:
        df[column_name + '(数字)'] = df[column_name].str.extract('(\d+)')
    return df

In [49]:
def df_drop(df: pd.DataFrame, column_names: list[str]) -> pd.DataFrame:
    for column_name in column_names:
        df.drop(column_name, axis=1, inplace=True)
    return df

In [50]:
def convert_date_and_time(df: pd.DataFrame) -> pd.DataFrame:
    df['発走日時'] = df['日付'].str.cat(df['発走時間'], sep=' ')
    df['発走日時'] = pd.to_datetime(df['発走日時'], format='%Y/%m/%d %H:%M')

    return df

In [51]:
def labelencoder(df: pd.DataFrame) -> pd.DataFrame:
    for column in LABELENCODER_COLUMNS:
        le = LabelEncoder()
        le.fit(df[column])
        df[column] = le.transform(df[column])

    return df

In [60]:
def convert_1st_2nd(df: pd.DataFrame) -> pd.DataFrame:
    conv_df = df['2車単'].str.extract('(\d+)>(\d+)', expand=True)
    df = pd.concat([df, conv_df], axis=1)

    return df

In [61]:
df_race_results: pd.DataFrame = get_df()
df_race_results = char_exclusion(df_race_results)
df_race_results = df_drop(df_race_results, RACE_RESULTS_CONV_HEADER)
df_race_results = convert_date_and_time(df_race_results)
df_race_results['日付'] = pd.to_datetime(df_race_results['日付'], format='%Y/%m/%d')
# df_race_results = labelencoder(df_race_results)
df_race_results = convert_1st_2nd(df_race_results)
df_race_results.columns = RACE_RESULTS_HEADER_RENAME
pd.to_pickle(df_race_results, 'race_results.pkl')
df_race_results = pd.read_pickle('race_results.pkl')
df_race_results

,日付,開催場所,グレード,レースグループ,レース名,発走時間,2車複,2車単,ワイド,3連複,3連単,ラウンド,距離,周回,発走日時,1着,2着
0,2021-07-11,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 初日特別選抜,20:25,3-4,3>4,3-4;3-5;4-5,3-4-5,3>4>5,12,2025,5,2021-07-11 20:25:00,3,4
1,2021-07-11,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 予選,19:54,1-7,7>1,1-7;5-7;1-5,1-5-7,7>1>5,11,2025,5,2021-07-11 19:54:00,7,1
2,2021-07-11,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 予選,19:25,1-4,4>1,1-4;4-7;1-7,1-4-7,4>1>7,10,2025,5,2021-07-11 19:25:00,4,1
3,2021-07-11,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 予選,18:59,2-3,2>3,2-3;2-6;3-6,2-3-6,2>3>6,9,2025,5,2021-07-11 18:59:00,2,3
4,2021-07-11,松山,FI,デイリースポーツ杯＆本田晴美杯,Ｓ級 予選,18:35,6-7,6>7,6-7;3-6;3-7,3-6-7,6>7>3,8,2025,5,2021-07-11 18:35:00,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,2021-02-08,奈良,FII,チャリロト杯,Ａ級 一般,13:00,1-5,5>1,1-5;3-5;1-3,1-3-5,5>1>3,5,1682,5,2021-02-08 13:00:00,5,1
308,2021-02-08,奈良,FII,チャリロト杯,Ａ級 チャレンジ選抜,12:32,2-5,5>2,2-5;5-7;2-7,2-5-7,5>2>7,4,1682,5,2021-02-08 12:32:00,5,2
309,2021-02-08,奈良,FII,チャリロト杯,Ａ級 チャレンジ選抜,12:04,3-6,6>3,3-6;4-6;3-4,3-4-6,6>3>4,3,1682,5,2021-02-08 12:04:00,6,3
310,2021-02-08,奈良,FII,チャリロト杯,Ａ級 チャレンジ一般,11:36,2-5,5>2,2-5;4-5;2-4,2-4-5,5>2>4,2,1682,5,2021-02-08 11:36:00,5,2
